In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/btech/mayank.kumar/Hritik/Multi-Task')
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PolymerSmilesTokenization import PolymerSmilesTokenizer
from sklearn.preprocessing import StandardScaler

/home/btech/mayank.kumar/miniconda3/envs/TransPolymer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('../data/practice.csv')
data

,smiles,Eea,Egb,Egc,Ei
0,*CC(*)C,0.4343,7.6997,6.5196,6.1850
1,*CC(*)F,0.8740,8.3099,7.6225,7.6332
2,*CC(*)(F)F,1.1415,-99.0000,7.1498,8.1531
3,*C(F)C(*)(F)F,1.5240,6.8451,7.6787,8.5986
4,*CCC(F)(F)C(*)(F)F,0.4489,9.1316,8.6882,9.0178
5,*CO*,0.9897,8.7732,6.6048,7.5826
6,*CCO*,0.3936,6.2206,5.5372,7.0344
7,*CC(O*)C(F)(F)F,0.7546,7.4158,6.7566,7.9431
8,*CCCO*,0.5672,7.4889,6.2459,6.9040
9,*Oc1ccc(Cc2ccc(*)cc2)cc1,1.1795,4.3787,4.9481,6.1428


In [3]:
tokenizer = PolymerSmilesTokenizer.from_pretrained("roberta-base", max_len=411)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'PolymerSmilesTokenizer'.


In [4]:
class Downstream_Dataset(Dataset):
    def __init__(self, dataset, tokenizer, max_token_len):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.max_token_len = max_token_len

    def __len__(self):
        self.len = len(self.dataset)
        return self.len

    def __getitem__(self, i):
        data_row = self.dataset.iloc[i]
        seq = data_row.iloc[0]
        prop = data_row.iloc[1]

        encoding = self.tokenizer(
            str(seq),
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return dict(
            seq = str(seq),
            input_ids=encoding["input_ids"].flatten(),
            attention_mask=encoding["attention_mask"].flatten(),
            prop=prop
        )

In [5]:
train_dataset = Downstream_Dataset(data, tokenizer, 411)

In [6]:
train_dataset[0]

{'seq': '*CC(*)C',
 'input_ids': tensor([   0, 3226,  347,  347, 1640, 3226,   43,  347,    2,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,   